# Phase 2 Project

## Business Understanding


Our chosen stakeholder is the real-estate agency Keller Williams, who's looking to expand into King County in Washington. They want an analytically supported strategy based on inferential and predictive analysis of the data available on the king county website. Our approach to formulating the business question was to first define our recommended strategy and formulate the business question around it. Accordingly, we formulated three questions that we wanted to answer using our data analysis and based our recommendations on those questions. 

### Recommended strategy 

We defined our strategy based on the volume metrics of the data and determined that the best way moving forward is to target sellers and buyers of houses that are in highest demand. We chose this strategy because we felt this approach would maximize your future potential revenue. This is based on the notion that a higher quantity of sales would result in more revenues than higher-value sales. 

### Business Question

Given our recommended strategy, the business question we formulated is :What types of houses are in most demand and where are they located?

## Data Understanding


The data that we used originally came from the King County website. It contains a good mix of categorical and numerical data. We wanted to focus on variables that corresponded to features that determine that demand of the any given house. 

### Importing Libraries

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
from yellowbrick.regressor import ResidualsPlot
import statsmodels.api as sm
from statsmodels.api import OLS

#SKLearn stuff
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

The libraries that we have imported are libraries that we used throughout this project, in data preparation and modeling sections.

In [10]:
df = pd.read_csv('../../data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


The table above illustrates the raw data set and how its formatted, while the table below lists all the columns in the data set and what data type the information in that column is. Furthermore, it also gives us an initial idea of the proportion of missing (null) values there are in each column. 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

## Data Preparation


Our approach to data preparation was systematic. First we dropped columns that that didn't have enough data to incorporate into our model and we also removed some extraneous outliers

#### Drops

We dropped the variable 'yr_renovated' because it was missing values for about 70% of our dataset. Furthermore, we dropped the variables 'id' and 'date' because we felt that those variables will be less reliable predictors. We also dropped an outlier from our data set; a house with 33 bedrooms. We also created a bedrooms bin column to use ahead.

In [12]:
df = df[df['bedrooms'] != 33]

In [13]:
#Making a categorical bedroom columns
df['bedrooms_bin']=pd.cut(df['bedrooms'], bins = [0,1,2,3,4,5,6,99], labels=['1BR','2BR','3BR','4BR','5BR','6BR','7+BR'])

In [14]:
df = df.drop(['id', 'date', 'yr_renovated','sqft_basement'], axis=1) 

#### Removing Outliers

When looking at removing outliers from our data set, we first used the numpy library to calculate our quantiles to determine which rows are the outliers. Based on the values we calculated for two numerical variables : 'price' and 'sqft_lot', we removed the corresponding rows. Furthermore, for the categorical variable 'grade' we performed the value_counts method in pandas to determine the counts of houses for each grade. Since the grades '3 Poor' and '12 Luxury' only have one record each, we excluded those values from our final processing data set as well.

In [15]:
price_q75, price_q25 = np.percentile(df['price'],[75,25])
intr_qrt = price_q75-price_q25
upper_price = price_q75 + (1.5*intr_qrt)
lower_price = price_q25 - (1.5*intr_qrt)
lower_price, upper_price
df = df[df['price'] <= upper_price]
df = df[df['price'] >= lower_price]

In [16]:
df['sqft_lot'].describe()

count    2.043800e+04
mean     1.458599e+04
std      4.001076e+04
min      5.200000e+02
25%      5.000000e+03
50%      7.500000e+03
75%      1.031675e+04
max      1.651359e+06
Name: sqft_lot, dtype: float64

In [19]:
sqft_lot_q75, sqft_lot_q25 = np.percentile(df['sqft_lot'],[75,25])
intr_qrt = sqft_lot_q75-sqft_lot_q25
upper_sqft_lot = sqft_lot_q75 + (1.5*intr_qrt)
lower_sqft_lot = sqft_lot_q25 - (1.5*intr_qrt)
lower_sqft_lot, upper_sqft_lot
df = df[df['sqft_lot'] <= upper_sqft_lot]
df = df[df['sqft_lot'] >= lower_sqft_lot]

In [20]:
df['grade'].value_counts()

7 Average        8188
8 Good           5152
6 Low Average    1839
9 Better         1800
10 Very Good      481
5 Fair            188
11 Excellent       57
4 Low              18
12 Luxury           1
3 Poor              1
Name: grade, dtype: int64

In [21]:
df = df[df['grade'] != '3 Poor']
df = df[df['grade'] != '12 Luxury']

### Helper Functions

These are a list of helper functions that we created to help us streamline our work flow and minimize repeating code 

In [22]:
def ohe(df, df2, column):    
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(drop="first", sparse=False, handle_unknown="error")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train,
                                     columns=col_names, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
        test = df2[[col]]
        encoded_test = ohe.transform(test)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_test = pd.DataFrame(encoded_test, 
                                    columns=col_names, index=df2.index)
        df2 = pd.concat([df2, encoded_test], axis=1)
    return df, df2, encoded_train.columns.tolist()

The function above creates a pipeline to perform two different transformations to test or train? data for any given input variable: First it utilizes the OneHotEncoder to take our categorical variables and encode them into a form that we can use in linear regression. The function takes in a training df, 'df', a test df, 'df2', and a column name, ['column_name']. It returns the training df and the test df concatenated with the new encoded columns. It also returns a list of the new encoded column names.  Secondly, it runs linear regression on encoded values by using .fit().

In [23]:
def scale_n_score(x,y,x2,y2):        
    scaler = StandardScaler()
    scaler.fit(x)
    X_train_scaled = scaler.transform(x)
    X_test_scaled = scaler.transform(x2)

    lr = LinearRegression()
    lr.fit(X_train_scaled,y_train)

    train_preds = lr.predict(X_train_scaled)
    test_preds = lr.predict(X_test_scaled)

    print('Training Score:',r2_score(y_train,train_preds))
    print('Test Score:',r2_score(y_test,test_preds))
    return X_train_scaled, X_test_scaled, train_preds, test_preds

The function above performs two actions; first it runs Standard Scaler, which scales all the values, in the test data and training data respectively, so that the output of our models are depicted in comparable units. Secondly, it scores the scaled data to get the R^2 results, a metric used in determining the efficacy of our model. 

In [24]:
def get_train_test_metrics(y_train,train_preds,
               y_test,test_preds
              ):
    """
    Prints different training and testing metrics, namely R2, MAE, MSE, RMSE
    """
    print("\nTraining Metrics:")
    print(f"R2: {r2_score(y_train, train_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")
    
    print("\nTesting Metrics:")
    print(f"R2: {r2_score(y_test, test_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

The function above uses the inputs of price and the predictions from our model to dtermine Mean Absolute Error (MAE), Mean Squared Error (MSE) and Root Mean Squared Error (RSME) for both the training and the testing data sets. Comparison of these values allow us to see how our model has performed. 

### The Train-Test Split

We set price as a dependent/target variable and every other variable in our data set as the independent/predictor variables. Then we performed a train/test split on the data. This will allow us to see the true performance of our model because we will be fitting our linear regression to our training data and then applying it to our testing data. The purpose of this is to treat the testing data as unseen data (data that our model hasn't experienced yet) and by comparing the training metrics between these two-data sets we can gain a better understanding of the validity of each model we create in the iterative process. 

In [25]:
y = df["price"]
X = df.drop("price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.25)

### Categorical Variables


The variables : 'bedroom_bin', 'view', 'zipcode', 'condition', and 'grade' are categorical variables. While the variables 'waterfront' is a binomial categorical variable. Each of these variables need to be converted into a format that can be modeled. 

#### Waterfront

Since this variable is a binomial categorical variable, OridinalEncoder was used to change this variable into a numeric one. Values denoted with "NAs" were determined to be "No" so those values were replaced as such.  

In [26]:
X_train['waterfront'].fillna('NO', inplace=True)
X_test['waterfront'].fillna('NO', inplace=True)

In [27]:
waterfront_train = X_train[['waterfront']]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_train)
encoder_waterfront.categories_[0]
waterfront_encoded_train = encoder_waterfront.transform(waterfront_train)
waterfront_encoded_train = waterfront_encoded_train.flatten()
X_train["waterfront"] = waterfront_encoded_train

In [28]:
waterfront_test = X_test[['waterfront']]
encoder_waterfront = OrdinalEncoder()
encoder_waterfront.fit(waterfront_test)
encoder_waterfront.categories_[0]
waterfront_encoded_test = encoder_waterfront.transform(waterfront_test)
waterfront_encoded_test = waterfront_encoded_test.flatten()
X_test["waterfront"] = waterfront_encoded_test

#### View 

The variable 'view' has 5 categories; NONE, AVERAGE, GOOD, FAIR, and EXCELLENT. The helper function we created above was use to encode this variable into a numeric one. We also replaced the string values so we could use it for comparative analysis to make our modeling process more efficient

In [29]:
X_train['view'].isna().sum()

45

In [30]:
X_train['view'].fillna("NONE", inplace=True)
X_test['view'].fillna("NONE", inplace=True)

In [31]:
X_train, X_test, encoded_view = ohe(X_train, X_test, ['view'])

In [32]:
# Change view rating to a numeric value for comparative purposes.
X_train['view'] = X_train['view'].map({'EXCELLENT':5,
                             'GOOD':4,
                             'AVERAGE':3,
                             'FAIR':2,
                             'NONE':1})

In [33]:
X_test['view'] = X_test['view'].map({'EXCELLENT':5,
                             'GOOD':4,
                             'AVERAGE':3,
                             'FAIR':2,
                             'NONE':1})

#### Condition 

We again replaced the string values in this column and the utilized the ohe function to perform the same encoding as above.